<a href="https://colab.research.google.com/github/LokeshDaga/ML_Assignment/blob/master/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Part 1**

Sol 1. OLS method

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
data = pd.DataFrame(pd.read_excel("/content/drive/My Drive/slr06.xls"))
X = data['X'].values
y = data['Y'].values
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=1)
mean_x = np.mean(X_train)
mean_y = np.mean(y_train)
covmat = np.cov(X_train,y_train)
m= covmat[0][1]/covmat[0][0]
c = mean_y - (m * mean_x)
print(m,c)
y_train_res = (m*X_train)+c
y_test_res = (m*X_test)+c
print("mean absolute error on train set is: ",mean_absolute_error(y_train,y_train_res))
print("mean absolute error on test set is: ",mean_absolute_error(y_test,y_test_res))

*** No CODEPAGE record, no encoding_override: will use 'ascii'
3.404887445482695 22.296085203742095
mean absolute error on train set is:  29.496246635464164
mean absolute error on test set is:  24.838142991133815


Sol 2. Gradient Descent Algorithm

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size=0.25,random_state=1)
theta_0,theta_1,alpha =0,0,0.0001
prev_theta_0 = prev_theta_1 = -1
n = float(len(X_train))
while (prev_theta_0!=theta_0 and prev_theta_1!=theta_1):
  y_predicted_train = theta_0 + (theta_1*X_train)
  prev_theta_0,prev_theta_1 = theta_0,theta_1
  cf_theta_0 = (1/n)*(np.sum(y_predicted_train - y_train))
  cf_theta_1 = (1/n)*(np.sum(X_train*(y_predicted_train-y_train)))
  theta_0 = theta_0 - (alpha*cf_theta_0)
  theta_1 = theta_1 - (alpha*cf_theta_1)
print(theta_0,theta_1)
y_predicted_train = theta_0 + (theta_1*X_train)
y_predicted_test = theta_0 + (theta_1*X_test)
y_predicted_val =  theta_0 + (theta_1*X_val)
print('MAE in training set:',mean_absolute_error(y_train,y_predicted_train))
print('MAE in validation set:',mean_absolute_error(y_val,y_predicted_val))
print('MAE in testing set: ',mean_absolute_error(y_test,y_predicted_test))

19.883112423697547 3.4230586885087346
MAE in training set: 28.995699850333967
MAE in validation set: 31.453019377642015
MAE in testing set:  24.028888510902316


**Part 2**

In [ ]:
from sklearn.datasets import load_boston
boston_dataset = load_boston(return_X_y=False)
#print(boston_dataset.keys())
#print(boston_dataset.DESCR)
boston = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
boston['MEDV'] = boston_dataset.target # Median value of owner-occupied homes in $1000s ( we have to predict it )
#boston.head(10)
data = boston.to_numpy() #converting DataFrames to numpy array
print("boston data shape (row,column) : ",data.shape)

#dividing data into training set , validation set and test set
sz = data[:,0].size
train_data = data[0:int(sz*0.6),:]
validate_data = data[int(sz*0.6) : int(sz*0.8)   , :]
test_data = data[int(sz*0.8)  : , :]
print("total_data : " ,sz)
print("train_data : " , train_data[:,0].size)
print("validation_data : " ,validate_data[:,0].size)
print("test_data : " ,test_data[:,0].size) 

#======================================================================

# gradientDescent function to calculate optimal theta
def gradientDescent(X, y, theta, alpha, num_iters):
    m = y.size  # number of training examples
    for i in range(num_iters):
        y_hat = np.dot(X, theta) # y_predicted (predicted housing price)
        theta = theta - alpha * (1.0/m) * np.dot(X.T, y_hat-y) 
    return theta


# adding column of ones to the data sets for theta0 ( i.e. the first parameter int theta )
n=train_data[:,0].size
ones=np.ones((n,1))
train_data = np.column_stack((ones,train_data)) # or like -> np.c_[ones, train_data]

n=validate_data[:,0].size
ones=np.ones((n,1))
validate_data = np.column_stack((ones,validate_data))

n=test_data[:,0].size
ones=np.ones((n,1))
test_data = np.column_stack((ones,test_data))



print("=========== GRADIENT DESCENT ALGORITHM STARTS : ==============")
nrow,ncol = data.shape
x = train_data[:,0:ncol]
y = train_data[:,ncol:]
theta = np.zeros((ncol,1))
alpha = 0.000001
iter = 100000
theta = gradientDescent(x,y,theta,alpha,iter)
print("theta from training set : ",theta)
y_predicted = np.dot(x,theta) 
mae = MAE(y_predicted , y)

# tuning the hyper-parameter (alpha) using validation data_set
values_of_alpha = [0.000003 , 0.000001, 0.0000003 , 0.00000001 , 0.00000003]
min_mae = mae
res_alpha = alpha
res_theta = theta
x = validate_data[:,0:ncol]
y = validate_data[:,ncol:]

for alpha in values_of_alpha: # finding the best value of alpha which gives minimum value of MAE on validation set
  theta = np.zeros((ncol,1))
  theta = gradientDescent(x,y,theta,alpha,iter)
  y_predicted = np.dot(x,theta) 
  mae = MAE(y_predicted , y)
  if mae < min_mae:
    res_alpha = alpha
    res_theta = theta
    min_mae = mae

theta = res_theta
print("theta after tuning alpha from validation set : ",theta)
 

# calculating accuracy on train data 
x = train_data[:,0:ncol]
y = train_data[:,ncol:]

y_predicted = np.dot(x,theta) 
mae = MAE(y_predicted , y)
print("MEAN ABSOLUTE ERROR ON TRAINING DATA : ",mae)


# calculating accuracy on validation data 
x = validate_data[:,0:ncol]
y = validate_data[:,ncol:]

y_predicted = np.dot(x,theta) 
mae = MAE(y_predicted , y)
print("MEAN ABSOLUTE ERROR ON VALIDATING DATA : ",mae)


# calculating accuracy on test data 
x = test_data[:,0:ncol]
y = test_data[:,ncol:]

y_predicted = np.dot(x,theta) 
mae = MAE(y_predicted , y)
print("MEAN ABSOLUTE ERROR ON TEST  DATA : ",mae)




boston data shape (row,column) :  (506, 14)
total_data :  506
train_data :  303
validation_data :  101
test_data :  102
=========== GRADIENT DESCENT ALGORITHM STARTS : ==============
theta from training set :  [[ 0.02277276]
 [ 0.07891455]
 [ 0.06918869]
 [-0.01713093]
 [ 0.02599327]
 [ 0.0184991 ]
 [ 0.44055185]
 [ 0.08587467]
 [-0.15999092]
 [ 0.17182027]
 [-0.00092437]
 [-0.15714102]
 [ 0.07412098]
 [-0.85012851]]
theta after tuning alpha from validation set :  [[ 0.04225091]
 [-0.12855066]
 [ 0.06273439]
 [-0.07674999]
 [ 0.10012007]
 [-0.00796301]
 [ 0.10373847]
 [ 0.08264848]
 [-0.23843666]
 [ 0.18461958]
 [ 0.004664  ]
 [ 0.483971  ]
 [ 0.0399667 ]
 [-1.02870376]]
MEAN ABSOLUTE ERROR ON TRAINING DATA :  6.227073327937616
MEAN ABSOLUTE ERROR ON VALIDATING DATA :  4.592752365421676
MEAN ABSOLUTE ERROR ON TEST  DATA :  6.030333031725009
